# Description
This notebook generates cards displaying relevant information such as the photos and annotations so they can be printen for usage in workshops. It depends on the CSV outputted by the convertJsonToCSV notebook. The standard settings outputs A4 PDFS containing 4 cards each using the ReportLab Python package (https://www.reportlab.com/dev/install/open_source_installation/)

Note: The cards are drawn based on pixel position, so when making changes it will take some time to get everything in the proper place

In [1]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.utils import ImageReader
from reportlab.lib.colors import HexColor
from reportlab.lib.colors import PCMYKColor, PCMYKColorSep, Color, black, blue, red
import os
import json
import time
import csv

In [2]:
fontsizeMapping = [6,7,8,9,10,11]
translation_mapping = {
        "Architecture":"Arkitektur",
        "Urban Environment": "Urbant miljø",
        "Objects": "Objekter",
        "Signs & symbols": "Tegn & symboler",
        "Urban Nature": "Bynatur",
        "Wind & Weather": "Vind og vejr",
        "Culture" : "Kultur",
        "Consumption" :"Forbrug",
        "People /community" :"Mennesker/fællesskaber",
        "Infrastucture" :"Infarstruktur",
        "Memories & associations": "Minder & associationer",
        "Atmosphere/Vibe":"Stemning",
        "Don't Know":"Ved ikke",
        "Other":"Andet"
}


# Note
Doublecheck the lines which contain photo[?] to see if the numbers correspond to the proper columns in the CSV. These might have changed if you made any changes in the JSONToCSV notebook. 

In [5]:
with open('dataoutput_test2.csv', newline='', encoding='utf8') as csvfile:
    photos = csv.reader(csvfile, delimiter=',')
    next(photos)
    j=0
    row_count = sum(1 for row in photos)
    totalPictures = row_count
    currentPhoto = 0
    pdfNr = 0
    for photo in photos:
        currentPhoto +=1
        annotation_categories = {
            "Architecture": 0,
            "Urban Environment": 0,
            "Objects": 0,
            "Signs & symbols": 0,
            "Urban Nature": 0,
            "Wind & Weather": 0,
            "Culture": 0,
            "Consumption": 0,
            "People /community": 0,
            "Infrastructure": 0,
            "Memories & associations": 0,
            "Atmosphere": 0,
            "Don't know": 0,
            "Other": 0 }
        #check if there is a slider answer, a few errors occured where these values did somehow not get stored.     
        if len(photo[9]) == 0:
            pass
        else:
            thumbnail_photo = ImageReader(photo[0])
            slider_answer = photo[9]
            mp_answer_string = photo[10]
            mp_answerList = mp_answer_string.split(',')
            danish_list = []
            for word in mp_answerList:
                word =  word.strip()
                if word in translation_mapping:
                    danish_list.append(translation_mapping[word])
                else:
                    danish_list.append(word)
            
            userId = photo[2]
            #UNIQUE CODE
            code = photo[78]
            
            if not os.path.exists("photocards/"):
                os.makedirs("photocards/")
                
            move_right = 0
            move_down = 0
            print(j)
            if j == 0:
                c = canvas.Canvas("photocards/"+str(pdfNr)+".pdf", pagesize=A4)
                c.setFont('Helvetica', 8)
                width, height = A4
                c.setFillColor(HexColor('#E3E3E3'))
                c.rect(0,0,600,850, fill=True, stroke=False)
                move_right = 0
                move_down = 0
           
            if j == 1:
                move_right = 280
                move_down = 0
            
            if j == 2:
                move_right = 0
                move_down = 420
            
            if j == 3:
                move_right = 280
                move_down = 420
                
            
            c.setFillColor(HexColor('#153725'))
            c.setFont('Helvetica-Bold', 8)
            c.setFontSize(15)
            c.drawImage(thumbnail_photo, 22+move_right, 570-move_down, width=240, height=240)
            
            #Check if there is location data or not.
            if len(photo[7]) == 0:
                c.drawString(22+move_right,825-move_down,code + '-?')
            else:
                c.drawString(22+move_right,825-move_down,code)

            c.setFontSize(12)
            c.drawString(22+move_right,550-move_down, "DINE ANNOTERINGER")


            c.setFont('Helvetica', 10)
            c.drawString(22+move_right,530-move_down, "I hvilken grad føler du dette er for dig?")

            #SLIDER BAR
            c.setFontSize(10)
            c.drawString(22+move_right,515-move_down, "Ikke for mig")
            c.drawString(228+move_right,515-move_down, "For mig")

            #DRAW NUMBERS ON SLIDER
            startStep = 93+move_right
            for i in range(1,6):
                if i == int(slider_answer):
                     c.setFont('Helvetica-Bold', 10)
                     c.setStrokeColor(HexColor('#48A675'))
                     c.setFillColor(HexColor('#48A675'))                         
                     c.setLineWidth(1)
                     c.line(startStep-1, 513-move_down, startStep+6, 513-move_down)
                else:
                     c.setFont('Helvetica', 10)
                     c.setFillColor(HexColor('#153725'))
                c.drawString(startStep,515-move_down,str(i))
                startStep += 26

            c.setFillColor(HexColor('#153725'))
            c.drawString(22+move_right,490-move_down,"Hvad fik dig til at tage billedet?")
            c.setFont('Helvetica-Bold', 10)
            c.setFillColor(HexColor('#48A675'))
            createString = ''
            maxStringLength = 48
            start_height = 477-move_down
            for i in range(len(danish_list)):
                print(danish_list[i])
                if len(createString) < maxStringLength and (len(createString) + len(danish_list[i])) <= maxStringLength:
                    createString = createString + danish_list[i] +', '
                    if i == len(danish_list)-1:
                        c.drawString(22+move_right,start_height,createString[:-2])
                else:
                    c.drawString(22+move_right,start_height,createString)
                    start_height = start_height-10
                    createString = danish_list[i] +', '
            
            if j == 3:
                j = 0
                c.save()
                pdfNr += 1
                break
                
            else:
                j += 1 
                if currentPhoto == totalPictures:
                    c.save()
            
            
                
                
                
                
                
  

1462


StopIteration: 